In [1]:
import pandas as pd
pdf19 = pd.read_csv('./data/pdf19.csv', encoding='euc-kr')
pdf19_input = pdf19.drop('학과', axis=1).to_numpy()
pdf19_target = pdf19['학과'].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(pdf19_input, pdf19_target, test_size=0.2, random_state=33)

In [5]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, random_state=33)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.7238095238095238
0.6415094339622641


----------------
<span style="color:yellow">6. 결정 트리 + 교차 검증, 그리드 서치<span>

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, random_state=33)

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
import numpy as np
np.mean(scores['test_score'])

0.6232946298984035

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=33)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
import numpy as np
np.mean(scores['test_score'])

0.6279202279202281

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.001, 0.002, 0.003, 0.004, 0.005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=33), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9543726235741445

In [16]:
gs.best_params_

{'max_depth': 10,
 'min_impurity_decrease': 0.0006000000000000001,
 'min_samples_split': 2}

In [15]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(3, 20, 1), 'min_samples_split': range(2, 100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=33), params, n_jobs=-1)
gs.fit(train_input, train_target)
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9581749049429658

In [17]:
np.max(gs.cv_results_['mean_test_score'])

0.6731494920174166

In [18]:
from scipy.stats import uniform, randint

In [31]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(3, 10), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1,20)}

In [35]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=33), params, n_iter=1000, n_jobs=-1, random_state=33)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=33),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024A3B9C8EB0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024A3B9C8580>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024A3B79DFA0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000024A3B9711F0>},
                   random_state=33)

In [36]:
gs.best_params_

{'max_depth': 6,
 'min_impurity_decrease': 0.0010024280592972577,
 'min_samples_leaf': 6,
 'min_samples_split': 16}

In [37]:
np.max(gs.cv_results_['mean_test_score'])

0.6803338171262701

In [39]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.6212121212121212